# Bug in Lake Salton

In [1]:
!pip uninstall -y eepackages

In [2]:
import os
from pathlib import Path
import sys

import ee


# Import debug version of eepackages, otherwise install latest
sys.path.append(str(Path.cwd().parent.parent.parent.absolute()) + os.sep + "ee-packages-py")

from eepackages.applications.waterbody_area import computeSurfaceWaterArea, extrapolate_JRC

In [3]:
ee.Initialize()

First we run the full algorithm to see where we have strange data points:

In [23]:
# 90686 - Salton
fid = 90554  # mead
waterbody = ee.Feature(ee.FeatureCollection("projects/global-water-watch/assets/reservoirs/quality_score_reservoirs").filter(ee.Filter.eq("fid", fid)).first())
export_time_start_filter = "2022-06-01"
export_time_start = "2022-06-01"
export_time_stop = "2022-09-01"
water_occurrence = (
    ee.Image("JRC/GSW1_3/GlobalSurfaceWater")
    .select("occurrence")
    .unmask(0)
    .resample("bicubic")
    .divide(100)
)

In [24]:
scale = ee.Feature(waterbody).geometry().area().sqrt().divide(200).max(10).getInfo()

missions = ["L4", "L5", "L7", "L8", "L9", "S2"]

water_area = computeSurfaceWaterArea(
    waterbody=ee.Feature(waterbody),
    start_filter=export_time_start_filter,
    start=export_time_start,
    stop=export_time_stop,
    scale=scale,
    waterOccurrence=ee.Image(water_occurrence),
    opt_missions=missions,
    quality_score_attributes=["qTh_15", "qTh_10", "qTh_5", "qTh_0"],
)

water_area = (
    ee.FeatureCollection(water_area)
    .filter(
        ee.Filter.And(
            ee.Filter.neq("p", 101),
            ee.Filter.gt("ndwi_threshold", -0.15),
            ee.Filter.lt("ndwi_threshold", 0.5),
            ee.Filter.lt("filled_fraction", 0.6),
        )
    )
    .sort("system:time_start")
)

properties = [
    "MISSION",
    "ndwi_threshold",
    "quality_score",
    "area_filled",
    "filled_fraction",
    "p",
    "system:time_start",
    "area",
]
properties_new = [
    "mission",
    "ndwi_threshold",
    "quality_score",
    "water_area_filled",
    "water_area_filled_fraction",
    "water_area_p",
    "water_area_time",
    "water_area_value",
]

water_area = (
    ee.FeatureCollection(water_area)
    .select(properties, properties_new, False)
    .set("scale", scale)
)


In [6]:
water_area = water_area.getInfo()

In [7]:
import pandas as pd
df: pd.DataFrame = pd.DataFrame(
    list(map(lambda feature: feature["properties"], water_area["features"]))
)

In [8]:
df["water_area_time"] = pd.to_datetime(df["water_area_time"], unit="ms", utc=True)

In [9]:
df

,mission,ndwi_threshold,quality_score,water_area_filled,water_area_filled_fraction,water_area_p,water_area_time,water_area_value
0,S2,-0.035257,0.207929,2.947854e+08,0.114502,0.96,2022-06-04 18:34:17.667000+00:00,2.610318e+08
1,S2,-0.019718,0.200483,2.966085e+08,0.583392,0.95,2022-06-04 18:34:32.227000+00:00,1.235695e+08
2,S2,0.053893,0.204050,2.864826e+08,0.131999,0.97,2022-06-09 18:34:11.876000+00:00,2.486672e+08
3,S2,0.055283,0.198426,2.901895e+08,0.590522,0.97,2022-06-09 18:34:26.432000+00:00,1.188262e+08
4,S2,0.039211,0.206147,2.890425e+08,0.134310,0.96,2022-06-14 18:34:20.376000+00:00,2.502211e+08
5,S2,0.055449,0.200106,2.943112e+08,0.597177,0.96,2022-06-14 18:34:34.935000+00:00,1.185554e+08
6,S2,0.069822,0.204145,2.697062e+08,0.089562,0.98,2022-06-19 18:34:14.170000+00:00,2.455506e+08
7,S2,0.116338,0.198286,2.447022e+08,0.530320,0.98,2022-06-19 18:34:28.715000+00:00,1.149317e+08
8,S2,0.179793,0.202241,2.555449e+08,0.137333,0.98,2022-06-24 18:34:22.247000+00:00,2.204500e+08
9,S2,0.242359,0.195822,7.546453e+07,0.062581,0.99,2022-06-24 18:34:25.432000+00:00,7.074191e+07


## Get Images
Obtain raw images

In [27]:
from datetime import datetime

from eepackages.assets import getImages, getMostlyCleanImages
from eepackages.utils import computeThresholdUsingOtsu
import geemap

In [28]:
images = getImages(waterbody.geometry(), {
    'resample': True,
    'filter': ee.Filter.date(export_time_start, export_time_stop),
    'missions': missions,
    'scale': scale * 10,
}).sort("system:time_start")

### Get Mostly Clean Images
Get mostly clean images based on quality score of green pixel

In [29]:
quality_score_attributes = ["qTh_15", "qTh_10", "qTh_5", "qTh_0"]
props = waterbody.getInfo()["properties"]
quality_score_threshold = None
if quality_score_attributes:
    for prop in quality_score_attributes:
        att = props.get(prop)
        if att:
            quality_score_threshold = att
            break

options = {
  # 'cloudFrequencyThresholdDelta': -0.15
   'scale': scale * 5,
   'cloud_frequency': props.get('cloud_frequency'),
   'quality_score_cloud_threshold': quality_score_threshold,
}

g = waterbody.geometry().buffer(300, scale)

images = getMostlyCleanImages(images, g, options).sort('system:time_start')

## Test steps water area algorithm

An individual image with an incorrect water area value is taken. Then we plot different steps in the algorithm to debug what is happening at each step.

First we can the correct image and call it `i`

In [30]:
# i = image
timestr = "2022-06-24"
filtered = images.filterDate(ee.Date(timestr), ee.Date(timestr).advance(1, "day"))
i = ee.Image(filtered.toList(1, 2).get(0))

In [31]:
# wanted time: 2022-06-24 18:34:25.432000+00:00
datetime.utcfromtimestamp(i.get('system:time_start').getInfo() * 1e-3)

datetime.datetime(2022, 6, 24, 18, 34, 25, 432000)

Then we execute the `get_water_area_singleImage` function step by step

In [32]:
fillPercentile = 50 # // we don't trust our prior
ndwiBands = ['green', 'swir']
waterMaxImage = ee.Image().float().paint(waterbody.buffer(150), 1)
t = i.get('system:time_start')
i = (i
    .updateMask(waterMaxImage)
    .updateMask(i.select('swir').min(i.select('nir')).gt(0.001)))

In [33]:
geom = ee.Feature(waterbody).geometry()
ndwi = i.normalizedDifference(ndwiBands)
th = computeThresholdUsingOtsu(ndwi, scale, geom, 0.5, 0.7, -0.2)
water = ndwi.gt(th)

print threshold value

In [34]:
th.getInfo()

0.24235901733239493

Function to extrapolate the JRC to from a maximum "trusted" water occurrence value to 1.0 to the center of the waterbody

In [35]:
def extrapolate_JRC(waterbody, scale, max_trusted_occurrence=0.97):
    water_occurrence = (
        ee.Image("JRC/GSW1_4/GlobalSurfaceWater")
        .select("occurrence")
        .mask(1)  # fixes JRC masking lacking
        .resample("bicubic")
        .divide(100)
    )

    # we don't trust water occurrence dataset when water level is below this (values > th)
    min_water = water_occurrence.gt(max_trusted_occurrence)

    # Calculate distance from water Occurence max
    dist = (
        min_water.Not()
        .fastDistanceTransform(150)
        .reproject(ee.Projection("EPSG:4326").atScale(100))
        .resample("bicubic")
        .sqrt()
        .multiply(100)
    )

    # calculate max distance for scaling to 1
    max_distance = dist.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=waterbody.geometry(),
        scale=scale,
        bestEffort=True,
    ).get("distance")

    # scale distance values from min_trusted_occurrence to 1
    extrap_scale = ee.Number(1).subtract(max_trusted_occurrence).divide(max_distance)
    water_occurrence_extrapolated = dist.multiply(extrap_scale).add(
        max_trusted_occurrence
    )

    return water_occurrence.where(
        water_occurrence.gt(max_trusted_occurrence), water_occurrence_extrapolated
    )

### Set different water occurrence for gap filling
Set what should be used as reference water occurrence

In [36]:
# waterOccurrence = water_occurrence
# waterOccurrence = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').select('water').mean()
waterOccurrence = extrapolate_JRC(waterbody, scale, max_trusted_occurrence=0.97)

Then run the rest of the algorithm

In [37]:
area = (ee.Image.pixelArea().mask(water)
.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': geom,
  'scale': scale
}).get('area'))

waterEdge = ee.Algorithms.CannyEdgeDetector(ndwi, 0.5, 0.7)
imageMask = ndwi.mask()

imageMask = imageMask.focal_min(ee.Number(scale).multiply(1.5), 'square', 'meters')
waterEdge = waterEdge.updateMask(imageMask)

p = waterOccurrence.mask(waterEdge).reduceRegion(**{
'reducer': ee.Reducer.percentile([fillPercentile]),
'geometry': geom,
'scale': scale
}).values().get(0)

In [38]:
print(p.getInfo())

0.9713280831671461


Check out if the 95th percentile value is the same as p, often an indication that we are using JRC's "maximum" p

In [39]:
p95 = waterOccurrence.mask(waterEdge).reduceRegion(**{
'reducer': ee.Reducer.percentile([95]),
'geometry': geom,
'scale': scale
}).values().get(0)

In [40]:
print(p95.getInfo())

0.9861588560732589


The JRC has a maximum p value often higher than 1.0, probably due to resampling?

In [41]:
pMax = waterOccurrence.mask(waterEdge).reduceRegion(**{
'reducer': ee.Reducer.max(),
'geometry': geom,
'scale': scale
}).values().get(0)

In [42]:
print(pMax.getInfo())

0.9926495033058123


In [43]:
p = ee.Algorithms.If(ee.Algorithms.IsEqual(p, None), 101, p)

waterFill = waterOccurrence.gte(ee.Image.constant(p)).updateMask(  # TODO
    water.unmask(0, False).Not()
)
nonWater = ndwi.lt(-0.15).unmask(0, False)
waterFill = waterFill.updateMask(nonWater.Not())

## Plot Debug Layers

To investigate each step of the algorithm, lots of debugging layers are printed together with the water area observed and the water area filled

In [47]:
Map = geemap.Map()
Map.addLayer(i, {"bands": ["red", "green", "blue"]}, "raw")
Map.addLayer(waterFill, {}, "waterFill")
Map.addLayer(waterOccurrence, {}, "water occurrence used for water fill")
Map.addLayer(water, {}, "water detected")
Map.addLayer(nonWater, {}, "very low NDWI levels")
Map.addLayer(waterEdge, {"palette": ["red", "blue"]}, "edge detected")
Map.centerObject(waterbody)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [48]:
fill = (
        ee.Image.pixelArea()
        .mask(waterFill)
        .reduceRegion(**{"reducer": ee.Reducer.sum(), "geometry": geom, "scale": scale})
        .get("area")
    )

In [49]:
print(f"fill is {fill.getInfo() / 1e6} km2")
print(f"area is {area.getInfo() / 1e6} km2")

fill is 183.85588258496094 km2
area is 70.7419107421875 km2
